In [ ]:
# Question 2: Logistic Regression
!pip install -U pandas
!pip install -U numpy
!pip install -U plotly==5.10.0

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

dataset = np.genfromtxt('../data/q2/q2test.csv', delimiter=',')

x1 = dataset[1:, 0].reshape(-1,1)
x2 = dataset[1:, 1].reshape(-1,1)
y_test = dataset[1:, 2].reshape(-1,1)
x0 = np.ones((x1.shape))
x_test = np.append(x0, x1, axis=1)
x_test = np.append(x_test, x2, axis=1)
n = x_test.shape[0]

In [ ]:
#Question 2.a - Sample million of points

#Data points given
data_points= 1000000
#Intercept term
x0 = np.ones((data_points,1)) 

#Samples
x1 = np.random.normal(3, 2, data_points).reshape(-1,1)
x2 = np.random.normal(-1, 2, data_points).reshape(-1,1)

#Create matrix
x = np.append(x0,x1,axis=1)
x = np.append(x, x2, axis=1)

#Sample epsilon error
eps = np.random.normal(0, np.sqrt(2), data_points).reshape(-1,1)

theta = np.array([[3], [1], [2]])

#Generate the value of Y (given X, parameterized by given Theta)
y = np.dot(x,theta) + eps

#Shuffle data
temp = np.append(x, y, axis=1)
np.random.shuffle(temp)

x = temp[:,0:3]
y = temp[:,-1:]

In [ ]:
#Question 2.b - Apply Stochastic gradient descent
import time

theta = np.zeros((3,1))
alpha = 0.001  
b_size = np.array([1, 100, 10000, 1000000])

# Prediction function: h(θ) = x^Tθ 
def predict(x, theta):
    return x.dot(theta)

# Cost function: J(θ) = 1/2m * Σ(y-h(θ))^2
def cost(x,y,theta, m):
    return (1/(2*m)) * np.sum((y - predict(x,theta))**2)

cost_0 = cost(x, y, theta, data_points)
print("Initial Cost value for the hypothesis with zero parameters={}".format(cost_0))


def cost_grd(x, y, theta, m):
    return (1/m) * (np.zeros((3,1))+  x.T.dot(x.dot(theta)-y))


# Stochastic descent function
def stochastic_gradient_descent(x, y, theta, alpha, r, threshold=10e-7):
    
    start = time.time()

    i=0
    c=0.0
    theta_hist = theta
    c_avg = np.array([cost(x[0], y[0], theta, 1)])

    while(True):
        i+=1
        count = 0
        c_init = cost(x, y,theta, data_points)
        for b in mini_batch:
            x_b = b[0]
            y_b = b[1]
            c += cost(x_b, y_b, theta, r)
            
            check = 10000 if r==1 else 100 if r==100 else 10 if r==10000 else 1
            if(count%check == 0 and count!=0):
                c /= check
                print("Current average cost = {}".format(c)) 
                c_avg = np.append(c_avg, c)
                c=0.0
            theta -= alpha * cost_grd(x_b, y_b, theta, r)
            theta_hist = np.append(theta_hist,theta,axis=1)
            count +=1
        c_final = cost(x, y, theta, data_points)
        if (abs(c_final - c_init) < threshold):
            print(c_final, c_init, c_final-c_init)
            break
    
    end = time.time()
        

    return theta, c_final, theta_hist, i, end-start



values = []
for r in b_size:
    #number of batches (sub batches stored in an array mini_batch)
    mini_batch = [(x[i:i+r,:], y[i:i+r]) for i in range(0, data_points, r)]
    theta, c_final, theta_hist, iterations, time = stochastic_gradient_descent(x, y, theta, alpha, r)
    print(f'Cost of the model is {c_final} with {iterations} iterations and {time} s')
    values.append((theta, c_final, theta_hist, iterations, time, r))



print(values)

In [ ]:
err_og = cost(x_test, y_test, np.array([[3],[1],[2]]), n)
print("The Test error on Original Hypothesis with theta = 0 is = {}".format(err_og))

for value in values:
    (theta, c_final, theta_hist, iterations, time, r) = value
    err_new = cost(x_test, y_test, theta, n)
    print("The Test error on Learned Hypothesis = {}, , for r = {}".format(err_new, r))

    print("The difference in the test error of original and learned hypothesis is = {}, for r = {}".format(err_new-err_og, r))

In [ ]:
from matplotlib.animation import FuncAnimation

for value in values:
    
    #Plot Graph
    fig = plt.figure(figsize=(15,10))
    ax = fig.add_subplot(111, projection='3d')
    ax.view_init(45, 20)
    
    (theta, c_final, theta_hist, iterations, time, r) = value
    t = theta_hist.T[0:40000:5,:] if r==1 else theta_hist.T[0::15,:] if (r==100) or (r==10000)  else theta_hist.T[0::30,:] 
    step = 1000 if r==1 else 20 if (r==100) or (r==10000)  else 10
    sc = ax.scatter([], [], [], marker='o', c='k')
    x_, y_, z_ = [], [], []
 
    def animate(i):
        x_.append(t[i,0])
        y_.append(t[i,1])
        z_.append(t[i,2])
        sc._offsets3d = (x_, y_, z_)
        return sc

    anim = FuncAnimation(fig, animate, frames=np.arange(0, t.shape[0]), interval=200, repeat_delay=3000, blit=False)
    anim.save('q1_'+str(r)+'c.gif', writer='pillow')
    plt.show()